# 1. Scrape Reviews

In [ ]:
python operations_controller.py scrape --excel ../database/establishments.xlsx

# 2. Unify Reviews (Incremental)

In [ ]:
# Unify all new reviews
python operations_controller.py unify

# Unify specific establishments
python operations_controller.py unify --establishments "id1,id2,id3"

# Quick mode (minimal output)
python operations_controller.py unify --quick

# Rebuild entire collection (use with caution!)
python operations_controller.py unify --rebuild

# 3. Show Statistics

python operations_controller.py stats

# 4. Scrape and Unify (Combined)

In [ ]:
# Full workflow
python operations_controller.py scrape-and-unify --excel ../database/establishments.xlsx

# With quick unification
python operations_controller.py scrape-and-unify --excel ../database/establishments.xlsx --quick-unify

# 5. Verbose Mode

In [ ]:
# Add -v or --verbose to any command
python operations_controller.py unify --verbose

# Usage Examples

## Daily Operations

In [ ]:
# Quick daily unification
python operations_controller.py unify --quick

# Full scrape and unify new establishments
python operations_controller.py scrape-and-unify --excel new_establishments.xlsx

## Maintenance Operations

In [ ]:
# Check database status
python operations_controller.py stats

# Full rebuild (when needed)
python operations_controller.py unify --rebuild

# Verbose troubleshooting
python operations_controller.py unify --verbose

## Targeted Operations

In [ ]:
# Process specific establishments only
python operations_controller.py unify --establishments "687a51385c7e5bb6b9c1a5d6,another_id"

## Programmatic Usage

In [ ]:
from database.incremental_unifier import IncrementalUnifier
from engine.operations_controller import OperationsController

# Use the unifier directly
unifier = IncrementalUnifier(log_to_file=False, log_level="WARNING")
unifier.initialize()
results = unifier.run_for_all_establishments(show_stats=False)
unifier.cleanup()

# Use the controller
controller = OperationsController(verbose=False)
success = controller.unify_reviews(quick=True)